## Chloe's Explore Notebook for the NLP Group Project

### Imports

In [26]:
import pandas as pd
import numpy as np

# Scraping
import requests
from bs4 import BeautifulSoup

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import nltk.sentiment

from wordcloud import WordCloud

# Regex
import re

# Time
from time import strftime

import unicodedata
import json
from pprint import pprint

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Custom Imports
import acquire 
from prepare import *

# Turn off pink boxes for demo
import warnings
warnings.filterwarnings("ignore")

# pd.set_option('display.max_colwidth', -1)

### Acquire

In [27]:
# acquire.scrape_github_data()

In [28]:
# df = pd.DataFrame(acquire.scrape_github_data(),columns=['repo','language','readme_contents'])
# df.head()


In [29]:
df = pd.read_json('repo_readmes.json')

In [30]:
df.shape

(960, 3)

In [31]:
original = pd.read_json('repo_readmes.json')
print(original)

                              repo    language  \
0                  bitcoin/bitcoin         C++   
1          bitcoinbook/bitcoinbook    AsciiDoc   
2                bitcoinj/bitcoinj        Java   
3                     bitcoin/bips    Wikitext   
4          bitcoinjs/bitcoinjs-lib  TypeScript   
..                             ...         ...   
955              spesmilo/electrum      Python   
956  bitcoin-wallet/bitcoin-wallet        Java   
957        etotheipi/BitcoinArmory         C++   
958    bitcoin-dot-org/Bitcoin.org        HTML   
959               jgarzik/cpuminer           C   

                                       readme_contents  
0    Bitcoin Core integration/staging tree\n=======...  
1    Code Examples: ![travis_ci](https://travis-ci....  
2    [![Github Build Status](https://github.com/bit...  
3    People wishing to submit BIPs, first should pr...  
4    # BitcoinJS (bitcoinjs-lib)\n[![Github CI](htt...  
..                                                 ...  


### Prepare

In [32]:
df.head()

,repo,language,readme_contents
0,bitcoin/bitcoin,C++,Bitcoin Core integration/staging tree\n=======...
1,bitcoinbook/bitcoinbook,AsciiDoc,Code Examples: ![travis_ci](https://travis-ci....
2,bitcoinj/bitcoinj,Java,[![Github Build Status](https://github.com/bit...
3,bitcoin/bips,Wikitext,"People wishing to submit BIPs, first should pr..."
4,bitcoinjs/bitcoinjs-lib,TypeScript,# BitcoinJS (bitcoinjs-lib)\n[![Github CI](htt...


In [33]:
# if you want to see the whole column contents:
# pd.set_option('display.max_colwidth', -1)

In [34]:
# how many language observations do we have?

df.language.value_counts(normalize = True)

Java          0.2
C++           0.2
AsciiDoc      0.1
HTML          0.1
C             0.1
Python        0.1
Wikitext      0.1
TypeScript    0.1
Name: language, dtype: float64

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 960 entries, 0 to 959
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             960 non-null    object
 1   language         960 non-null    object
 2   readme_contents  960 non-null    object
dtypes: object(3)
memory usage: 30.0+ KB


Split data since we will be modeling. 

In [46]:
# SPLIT
from sklearn.model_selection import train_test_split
# Test set is .2 of original dataframe
train, test = train_test_split(df, test_size = .2, random_state=123, stratify= df.language)
# The remainder is here divided .7 to train and .3 to validate
train, validate = train_test_split(train, test_size=.3, random_state=123, stratify= train.language)
train.shape, validate.shape, test.shape

((537, 3), (231, 3), (192, 3))

In [47]:
# combine all contents in single string by language

java_words = basic_clean(' '.join(train[train.language == 'Java'].readme_contents))
c_plus_plus__words = basic_clean(' '.join(train[train.language == 'C++'].readme_contents))
wiki_words = basic_clean(' '.join(train[train.language == 'Wikitext '].readme_contents))
type_words = basic_clean(' '.join(train[train.language == 'TypeScript'].readme_contents))
c_words = basic_clean(' '.join(train[train.language == 'C'].readme_contents))
ascii_words = basic_clean(' '.join(train[train.language == 'AsciiDoc'].readme_contents))
python_words = basic_clean(' '.join(train[train.language == 'Python'].readme_contents))
html_words = basic_clean(' '.join(train[train.language == 'HTML'].readme_contents))
all_words = basic_clean(' '.join(train.readme_contents))

In [48]:
# all_words

### Explore

In [49]:
# represent contents as word frequencies

java_freq = pd.Series(java_words.split()).value_counts()
c_plus_plus_freq = pd.Series(c_plus_plus__words.split()).value_counts()
wiki_freq = pd.Series(wiki_words.split()).value_counts()
type_freq = pd.Series(type_words.split()).value_counts()
c_freq = pd.Series(c_words.split()).value_counts()
ascii_freq = pd.Series(ascii_words.split()).value_counts()
python_freq = pd.Series(python_words.split()).value_counts()
html_freq = pd.Series(html_words.split()).value_counts()
all_freq = pd.Series(all_words.split()).value_counts()

In [50]:
# concat all frequencies together into a dataframe

word_counts = pd.concat([java_freq, c_plus_plus_freq, wiki_freq, type_freq, c_freq, ascii_freq, python_freq, html_freq, all_freq], axis=1).fillna(0).astype(int)
word_counts.columns = ['java', 'c_plus_plus', 'wiki', 'typescript', 'c', 'asciidoc', 'python', 'html', 'all']
word_counts.head()

,java,c_plus_plus,wiki,typescript,c,asciidoc,python,html,all
0,0,0,0,162,0,0,0,0,215
1,0,0,0,54,0,54,0,0,161
10,0,0,0,0,0,54,0,0,54
11,370,0,0,0,0,54,0,0,424
12,0,0,0,0,0,54,0,0,54


- What are the most frequently occuring words? 

- Are there words that uniquely identify with a certain language's repos? 


In [51]:
# sort by 'all'

word_counts.sort_values('all', ascending=False).head(20)

,java,c_plus_plus,wiki,typescript,c,asciidoc,python,html,all
the,2477,2048,0,2106,54,1350,918,432,9915
a,847,474,0,3132,54,972,216,324,6602
and,1263,1397,0,2376,108,594,162,108,6538
standard,0,0,0,0,0,0,0,0,6466
to,1104,1122,0,1458,108,702,486,162,5460
stylebackgroundcolor,0,0,0,0,0,0,0,0,5300
for,911,1027,0,918,108,216,162,54,5092
of,526,593,0,864,0,648,162,216,3751
is,318,1091,0,810,162,486,486,54,3513
applications,52,0,0,0,0,0,0,0,3391
